In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input, Activation
from tensorflow.keras.activations import relu
from tensorflow.keras.applications import InceptionV3, DenseNet121, MobileNetV2
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import utils
from tensorflow.keras import Input
from sklearn.metrics import confusion_matrix
from sklearn.utils import resample
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import zipfile
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from PIL import Image
import cv2
import random 
import gc
from tensorflow.keras.backend import clear_session
import os

%matplotlib inline 

In [ ]:
# разархивируем архив Numpy
train = np.load('../input/university-of-digital-technologies3/train_pneumonia.npz')
val = np.load('../input/university-of-digital-technologies3/val_pneumonia.npz')
test = np.load('../input/university-of-digital-technologies3/test_pneumonia.npz')

In [ ]:
# Проверим какие есть разделы в файле.
train.files

In [ ]:
# Извлекаем выборки.
X_train = train['X_train']
Y_train = train['Y_train']
Y_train_cat = train['Y_train_cat']

X_val = val['X_val']
Y_val = val['Y_val']
Y_val_cat = val['Y_val_cat']

X_test = test['X_test']

In [ ]:
# Добавляем дополнительное измерение необходимое для обучения модели.
X_train_c = np.expand_dims(X_train, axis = 3)
print(X_train_c.shape)
X_test_c = np.expand_dims(X_test, axis = 3)
print(X_test_c.shape)
X_val_c = np.expand_dims(X_val, axis = 3)
print(X_val_c.shape)

In [ ]:
# Проверяем размерность меток.
print(Y_train.shape)
print(Y_val.shape)

In [ ]:
# Проверим сбалансированность тренировачной выборки.
print(len(np.where(Y_train == 0.)[0]))
print(len(np.where(Y_train == 1.)[0]))

In [ ]:
# Выведим два случайных снимка из каждой каткгории.
fig, ax = plt.subplots(1, 2, figsize=(14,6))
fig.suptitle('Снимки болного и здорового.', fontsize=15)
# Уберем метки осй и деления.
ax[0].set_xticks([])
ax[0].set_yticks([])
ax[1].set_xticks([])
ax[1].set_yticks([])
ax[0].imshow(X_train[np.random.choice(np.where(Y_train == 0.)[0])], cmap='gray')
ax[0].set_title('Здоровый')
ax[1].imshow(X_train[np.random.choice(np.where(Y_train == 1.)[0])], cmap='gray')
ax[1].set_title('Больной')

plt.show()

In [ ]:
# Переводим серую картинку в RGB, для создания 3 измерений вместо 1.
X = np.expand_dims(cv2.cvtColor(X_train_c[0], cv2.COLOR_GRAY2RGB), 0)
for i in range(1, X_train_c.shape[0]):
    X = np.append(X, np.expand_dims(cv2.cvtColor(X_train_c[i], cv2.COLOR_GRAY2RGB), 0), axis=0)
print(X_train_c.shape)
print(X.shape)

In [ ]:
gc.collect()

In [ ]:
# Переводим серую картинку в RGB, для создания 3 измерений вместо 1.
X_v = np.expand_dims(cv2.cvtColor(X_val_c[0], cv2.COLOR_GRAY2RGB), 0)
for i in range(1, X_val_c.shape[0]):
    X_v = np.append(X_v, np.expand_dims(cv2.cvtColor(X_val_c[i], cv2.COLOR_GRAY2RGB), 0), axis=0)
print(X_val_c.shape)
print(X_v.shape)

In [ ]:
gc.collect()

In [ ]:
# Переводим серую картинку в RGB, для создания 3 измерений вместо 1.
predict_test = np.expand_dims(cv2.cvtColor(X_test_c[0], cv2.COLOR_GRAY2RGB), 0)
for i in range(1, X_test_c.shape[0]):
    predict_test = np.append(predict_test, np.expand_dims(cv2.cvtColor(X_val_c[i],
                                                                       cv2.COLOR_GRAY2RGB), 0), axis=0)
print(X_test_c.shape)
print(predict_test.shape)

In [ ]:
gc.collect()

In [ ]:
# Устанавливаем необходимую размерность изображения.
IMG_SIZE = 192
BATCH_SIZE = 64
EPOCHS = 20

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, min_lr=0.0000000001, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model.hdf5', monitor='val_loss',
                                                save_weights_only=True,
                                                save_best_only=True, verbose=1)

In [ ]:
# Создаём генератор изображений.
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [ ]:
clear_session()
# загружаем предобученную сеть, убираем из исходной модели полносвязные слои
baseModel = MobileNetV2(weights="imagenet", include_top=False,
    input_tensor=Input(shape=(IMG_SIZE, IMG_SIZE, 3)))

# создаём полносвязные слои на выходе вместо убранных
# из базовой модели
headModel = baseModel.output
headModel = MaxPooling2D()(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(10, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(1, activation="sigmoid")(headModel)

# Cобираем финальную модель, которую хотим тренировать.
model = Model(inputs=baseModel.input, outputs=headModel)

# Проходим по слоям и "замораживаем" возможность изменения весов так,
# чтобы веса менялись лишь у добавленных слоёв.
for layer in baseModel.layers:
    layer.trainable = False

In [ ]:
# Удаляем все предыдущие сохранённые веса.
# for name in os.listdir():
#     if name.endswith('.hdf5'):
#         os.remove(f"./{name}")
        
# os.listdir()

In [ ]:
# Компилируем модель.
LR = 0.001
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate = LR), metrics=["accuracy"])

# тренируем нашу нейросеть
history = model.fit(
    datagen.flow(X, Y_train, batch_size=BATCH_SIZE),
#     steps_per_epoch=len(X) // BATCH_SIZE,
    validation_data=(X_v, Y_val),
#     validation_steps=len(X_v) // BATCH_SIZE,
    epochs=100,
    callbacks=[checkpoint, reduce_lr],
    verbose=1)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model_25.hdf5',
                                                monitor='val_loss', save_weights_only=True,
                                                save_best_only=True, verbose=1)

In [ ]:
# Разблокируем 25% весов модели и уменьшим уровень обучения, для более точной тренировки модели.
model.load_weights('./model.hdf5')
for layer in baseModel.layers[int(len(baseModel.layers)*0.75):]:
    layer.trainable = True
# len(model.trainable_variables)
# LR = 0.0001

In [ ]:
# Компилируем модель.
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate = LR), metrics=["accuracy"])

# Тренируем нашу нейросеть.
history = model.fit(
    X, Y_train, batch_size=BATCH_SIZE,
    steps_per_epoch=len(X) // BATCH_SIZE,
    validation_data=(X_v, Y_val),
    validation_steps=len(X_v) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, reduce_lr],
    verbose=1)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model_50.hdf5', monitor='val_loss', save_weights_only=True,
                                                save_best_only=True, verbose=1)

In [ ]:
# Разблокируем 50% весов модели и уменьшим уровень обучения, для более точной тренировки модели.
model.load_weights('./model_25.hdf5')

for layer in baseModel.layers[int(len(baseModel.layers)*0.5):]:
    layer.trainable = True
# len(model.trainable_variables)
LR = 0.0005

In [ ]:
# Компилируем модель.
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate = LR), metrics=["accuracy"])

# Тренируем нашу нейросеть.
history = model.fit(
    datagen.flow(X, Y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X) // BATCH_SIZE,
    validation_data=(X_v, Y_val),
    validation_steps=len(X_v) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, reduce_lr],
    verbose=1)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model_75.hdf5', monitor='val_loss', save_weights_only=True,
                                                save_best_only=True, verbose=1)

In [ ]:
# Разблокируем 75% весов модели и уменьшим уровень обучения, для более точной тренировки модели.
model.load_weights('./model_50.hdf5')

for layer in baseModel.layers[int(len(baseModel.layers)*0.25):]:
    layer.trainable = True
# len(model.trainable_variables)
LR = 0.0001

In [ ]:
# Компилируем модель.
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate = LR), metrics=["accuracy"])

# Тренируем нашу нейросеть.
history = model.fit(
    datagen.flow(X, Y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X) // BATCH_SIZE,
    validation_data=(X_v, Y_val),
    validation_steps=len(X_v) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, reduce_lr],
    verbose=1)

In [ ]:
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model_100.hdf5', monitor='val_loss', save_weights_only=True,
                                                save_best_only=True, verbose=1)

In [ ]:
# Разблокируем все веса модели и уменьшим уровень обучения, для более точной тренировки модели.
model.load_weights('./model_75.hdf5')

for layer in baseModel.layers:
    layer.trainable = True
# len(model.trainable_variables)
LR = 0.00001
EPOCHS = 20

In [ ]:
# Компилируем модель.
model.compile(loss="binary_crossentropy", optimizer=Adam(learning_rate = LR), metrics=["accuracy"])

# Тренируем нашу нейросеть.
history = model.fit(
    datagen.flow(X, Y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=len(X) // BATCH_SIZE,
    validation_data=(X_v, Y_val),
    validation_steps=len(X_v) // BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[checkpoint, reduce_lr],
    verbose=1)

In [ ]:
model.load_weights('./model.hdf5')

In [ ]:
model.evaluate(X_v, Y_val)

In [ ]:
y_pred = model.predict(predict_test)
y_pred.shape

In [ ]:
# переводим данные в формат 0 - 1
y = (y_pred > .6).astype('float32')
y = y.reshape(y.shape[0])
y

In [ ]:
# создаём датафрейм в нужном формате
submission = pd.DataFrame({"Id":range(1,len(y_pred)+1),"Label":y})
submission.head()

In [ ]:
# сохраняем его как csv
submission = submission.to_csv('./submission.csv', sep=',', index=False, header=True)

#### При трансферном обуении на сетях VGG16, InceptionV3, MobileNetV2, DenseNet121 - получаем хороший прирост на валидационной выборке, при этом на тестовой выюорке показывает максимум 86%, что намного хуже, чем на обученной простой сети с 2 свёртками(93.5%)(((

# Создаём обычную сеть.

In [ ]:
# Устанавливаем необходимую размерность изображения.
IMG_SIZE = 192
BATCH_SIZE = 32
EPOCHS = 20

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                                 patience=5, min_lr=0.0000000001, verbose=1)
checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath='model.hdf5', monitor='val_loss', save_weights_only=True,
                                                save_best_only=True, verbose=1)

In [ ]:
# Создаем пустой генератор для избегания переполнения памяти.
datagen = ImageDataGenerator()

In [ ]:
class MyModel(Model):

    def __init__(self, do=0.1):
        super().__init__()
        self.conv1 = Conv2D(16, 3, padding = 'same')
        self.conv2 = Conv2D(32, 3, padding = 'same')
        self.conv3 = Conv2D(128, 3, padding = 'same')
        self.flatten = Flatten()
        self.dense = Dense(32, activation="relu")
        self.out = Dense(1, activation="sigmoid")
        self.act = Activation('relu')
        self.pool = MaxPooling2D()
        self.drop_layer = Dropout(do)
        self.batch = BatchNormalization()
        

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.act(x)
        # x = self.batch(x)
        x = self.pool(x)
        # x = self.drop_layer(x)
        x = self.conv2(x)
        x = self.act(x)
        # x = self.batch(x)
        x = self.pool(x)
        # x = self.drop_layer(x)
        x = self.conv3(x)
        x = self.act(x)
        # x = self.batch(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.dense(x)
        # x = self.batch(x)
        # X = self.drop_layer(x)
        return self.out(x)

In [ ]:
# Создаем модель.
clear_session()
model = MyModel(do=0.5)
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, IMG_SIZE, IMG_SIZE, 1))
model.summary()

In [ ]:
gc.collect()

In [ ]:
history = model.fit(datagen.flow(X_train_c, Y_train, batch_size=64), epochs=15,
                    validation_data=(X_val_c, Y_val), verbose=1)#, callbacks=[checkpoint, reduce_lr])

In [ ]:
model.evaluate(X_val_c, Y_val)

In [ ]:
# Выводим график точности на обучающей выборке
# label - имя графика в легенде
plt.figure(figsize=(10, 8))
plt.plot(history.history['accuracy'], 
         label='Тестовая выборка.')

# Выводим график точности на проверочной выборке
plt.plot(history.history['val_accuracy'], 
         label='Валидационная выборка.')

# Выводим подписи осей
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')

# Выводим легенду
plt.legend()
plt.show()

In [ ]:
y_pred = model.predict(X_test_c)
y_pred.shape

In [ ]:
# переводим данные в формат 0 - 1
y = (y_pred > .5).astype('float32')
y = y.reshape(y.shape[0])
y

In [ ]:
# создаём датафрейм в нужном формате
submission = pd.DataFrame({"Id":range(1,len(y_pred)+1),"Label":y})
submission.head()

In [ ]:
# сохраняем его как csv
submission = submission.to_csv('./submission.csv', sep=',', index=False, header=True)